# Solving Sudoku

The primary description of this coursework is available on the CM20252 Moodle page. This is the Jupyter notebook you must complete and submit to receive marks. 

** You must follow all instructions given in this notebook. **

Restart the kernel and run all cells before submitting the notebook. This will guarantee that we will be able to run your code for testing.

Remember to save your work regularly. 

## Getting Started

You already know that you will be writing a Sudoku solver. You need to implement your solver in Python in this notebook. You can use any of the appropriate problem-solving techniques discussed in the lectures. You are encouraged to modify the basic algorithms. Be creative. 

You will be given Sudoku puzzles that either have a single solution or no solution. You will need to identify the solution, if there is one.

Below is a sample puzzle along with its solution. 

<img src="images/Sudoku_unsolved.png" style="width: 200px;"/>
<img src="images/Sudoku_solved.png" style="width: 200px;"/>


## Sample Sudokus

You can test your code on a set of 20 sample Sudoku puzzles. This set is similar to the test set that will be used to assess your work. 

The following code will load 20 Sudoku puzzles and their solutions into two `20x9x9` numpy arrays. Empty cells are designated by zeros. 

In [1]:
import numpy as np

# Load sudokus
sudokus = np.load("data/sudokus.npy")
print("Shape of one sudoku array:", sudokus[0].shape, ". Type of array values:", sudokus.dtype)

# Load solutions
solutions = np.load("data/sudoku_solutions.npy")
print("Shape of one sudoku solution array:", solutions[0].shape, ". Type of array values:", solutions.dtype, "\n")

# Print the first sudoku...
print("Sudoku #1:")
print(sudokus[0], "\n")

# ...and its solution
print("Solution of Sudoku #1:")
print(solutions[0])

Shape of one sudoku array: (9, 9) . Type of array values: float64
Shape of one sudoku solution array: (9, 9) . Type of array values: float64 

Sudoku #1:
[[0. 0. 4. 0. 8. 3. 0. 0. 2.]
 [0. 5. 1. 0. 0. 4. 3. 0. 0.]
 [0. 0. 0. 0. 9. 6. 7. 1. 0.]
 [1. 2. 0. 8. 0. 0. 0. 0. 6.]
 [0. 4. 0. 0. 0. 0. 5. 0. 0.]
 [8. 3. 0. 6. 0. 7. 9. 0. 0.]
 [0. 6. 0. 3. 0. 9. 0. 4. 0.]
 [0. 0. 7. 0. 0. 0. 2. 0. 5.]
 [0. 9. 0. 0. 5. 0. 8. 0. 3.]] 

Solution of Sudoku #1:
[[9. 7. 4. 1. 8. 3. 6. 5. 2.]
 [6. 5. 1. 2. 7. 4. 3. 8. 9.]
 [2. 8. 3. 5. 9. 6. 7. 1. 4.]
 [1. 2. 9. 8. 3. 5. 4. 7. 6.]
 [7. 4. 6. 9. 1. 2. 5. 3. 8.]
 [8. 3. 5. 6. 4. 7. 9. 2. 1.]
 [5. 6. 8. 3. 2. 9. 1. 4. 7.]
 [3. 1. 7. 4. 6. 8. 2. 9. 5.]
 [4. 9. 2. 7. 5. 1. 8. 6. 3.]]


## Your code ##

Define a function called `sudoku_solver()` that takes **one** Sudoku puzzle (a $9 \times 9$ numpy array) as input and returns the solved Sudoku as a $9 \times 9$ numpy array. Note that the test set may contain invalid Sudokus, that is, Sudokus with no solution. In such a case, your function should return a $9 \times 9$ numpy array whose values are all equal to `-1`.  

You may use more than one cell to write your code (but this is not necessary).

In [2]:
######################################################### AI CWK 1 #########################################################

"""
Implementation of the four way linked list objects in the dancing links paper (Knuth, 2000).

Knuth, D.E., 2000. Dancing links. arXiv preprint cs/0011047.


All Code is my own work, however the algorithm's from Knuths paper have been replecated.  Furthermore the eact cover sudoku
matrix is a replica of one done by Bob Hanson, however I nonetheless generated it using my own code.

"""

########################################################## Imports #########################################################

import numpy as np
import time

########################################################## Classes #########################################################

class DataObj:
    """
    A very simple object to represent each node in the 4-way linked list.  It has 5 peices of information: u is the link to
    the above data; d for down; l for left; r for right; c for the link to the column header (also a dataObj); name for 
    debugging and calling up column headers by reference; and size which is the number of nodes in a column.
    """
    def __init__(self, name=None, row=None):
        """
        This is to initialise the object with a name.  The name is nesecarry for the columns.
        """
        self.name = name # Used for debugging.
        self.row = row # Used for printing the solution.
    
    def setLinks(self, l=None, r=None, u=None, d=None, c=None):
        """
        This sets all of the links for the object.  This is a seperate function as all links must be initialised for a list
        before the links can be set.
        """
        self.l = l # Left link.
        self.r = r # Right link.
        self.u = u # Above link.
        self.d = d # Below link.
        self.c = c # Header link.
    
    def setSize(self, size=None):
        """
        This is a seperate function to set the size only of the object is a column header.
        """
        self.size = size



class ListObj():
    """
    This is the class which represents the whole linked node structure.  It is sort of like an array but not an array.
    In order to store the nodes in a sensible manner the exact cover array of ones and zeros is fed in, this is then used
    generate another array with data type "DataObj" with an extra column and row.  This array matches one to one with the
    exact cover array with ones becoming DataObjs and zeros being set to None, this provides an effecient and logical set
    up, which facilitates simplistic automated linking of the nodes.
    """

    def __init__(self, array):
        """
        Generates the nodes array from the input exact covr array.  self.nodes is the key data structure that is operated
        on.
        """
        self.nodes = self.initialiseNodes(array) # Initialises all nodes
        self.numRows = np.shape(self.nodes)[0]
        self.numCols = np.shape(self.nodes)[1]
        self.setLinks() # sets the links for the nodes
        self.setSize(array) # sets the size for the headers.

    def initialiseNodes(self, array):
        """
        This initialises all nodes in the node array.  Note that this does not actually add any information to the nodes.
        """
        numRows = np.shape(array)[0] # Number of rows in the passed arrays.
        numCols = np.shape(array)[1] # Number of columns in the passed array.

        nodes = np.empty((numRows+1,numCols+1), dtype = DataObj)
        # Initialises the array containing the nodes.
        nodes[0,0] = DataObj("r-0-0", row=0)
        # Initialises the root node.
        for i in range(numCols):
            nodes[0,i+1] = DataObj("c-0-"+str(i+1), row=0)
        # Initialises the column headers.
        for i in range(numRows):
            for j in range(numCols):
                if array[i,j] == True:
                    nodes[i+1,j+1] = DataObj("n-"+str(i+1)+"-"+str(j+1), row=i+1)
        # Initialises the nodes.
        return nodes

    def findNearest(self, index, direction):
        """
        This function takes a given index of a node and a direction and finds the nearest node in that direction.  This is
        used to generate a single link for a node.
        """
        if direction == "u": # Up direction.
            while True: # This loops until an object is found.
                if index[0] == 0: # This is a special condition if the loop reaches the START of the array.
                    index = (self.numRows-1,index[1]) # Takes the index to the END of the array.
                else:
                    index = (index[0]-1,index[1]) # Moves the search index up.
                if self.nodes[index] != None: # If a DataObj is found...
                    return self.nodes[index] # ...returns the DataObj.
        if direction == "d": # Down direction.
            while True:
                if index[0] == (self.numRows-1): # This is a special condition if the loop reaches the END of the array.
                    index = (0,index[1]) # Takes the index to the START of the array.
                else:
                    index = (index[0]+1,index[1])
                if self.nodes[index] != None:
                    return self.nodes[index]
        if direction == "l": # Left direction.
            while True:
                if index[1] == 0:
                    index = (index[0],self.numCols-1)
                else:
                    index = (index[0],index[1]-1)
                if self.nodes[index] != None:
                    return self.nodes[index]
        if direction == "r": # Right direction.
            while True:
                if index[1] == self.numCols-1:
                    index = (index[0],0)
                else:
                    index = (index[0],index[1]+1)
                if self.nodes[index] != None:
                    return self.nodes[index]

    def setLinks(self):
        """
        This sets all of the links for the nodes, by iterating over every element and running findNearest() if the element
        is a node.
        """
        for i in range(self.numRows):
            for j in range(self.numCols):
                if self.nodes[i,j] != None:
                    self.nodes[i,j].setLinks(l=self.findNearest((i,j), "l"), # Left
                                             r=self.findNearest((i,j), "r"), # Right
                                             u=self.findNearest((i,j), "u"), # Up
                                             d=self.findNearest((i,j), "d"), # Down
                                             c=self.nodes[0,j]) # The column header is always in the same position.

    def setSize(self, array):
        """
        Sets the sizes for the column headers by iterating through the first row of self.nodes and summing the columns of
        the input exact cover array.  Although summing the nodes could be done, this method allows the use of the default
        numpy sum fucntion and aray slicing which is very fast.
        """
        for i in range(self.numCols-1):
            self.nodes[0,i+1].setSize(np.sum(array[:,i]))

    def printNodes(self, link = None):
        """
        Prints the array names for diagnostics into a representation of the actual array - sooo pretty.  The function is set
        up such that it can display the name of the node above that position etc.  This is so that the links can also be
        investigated.
        """
        for i in range(np.shape(self.nodes)[0]):
            row = ""
            for j in range(np.shape(self.nodes)[1]):
                dataObj = self.nodes[i,j]
                if dataObj != None:
                    if link == None:
                        row = row+(dataObj.name)+'|'
                    if link == "u":
                        row = row+(dataObj.u.name)+'|'
                    if link == "d":
                        row = row+(dataObj.d.name)+'|'
                    if link == "l":
                        row = row+(dataObj.l.name)+'|'
                    if link == "r":
                        row = row+(dataObj.r.name)+'|'
                else:
                    row = row+"     |"
            print(row)

    def printSize(self):
        """
        Prints the colum sizes and names, also for diagnostics.
        """
        row = ""
        for i in range(self.numCols-1):
            row = row+str(self.nodes[0,i+1].size)+" "+self.nodes[0,i+1].name+"|"
        print(row)



class Solver:
    """
    This is the solver class implementing Dancing Links by Knuth.  The solver is for the exact cover problem and is sperate
    from the suduko solver which merely wraps this up.
    """

    def __init__(self, array):
        """
        Takes the exact cover array and creates a ListObj
        """
        self.listObj = ListObj(array)
        self.maxColumnSize = self.listObj.numRows
        
        self.oListSize = 100
        # This is the expected size of the oList, by pre-allocating the algorithm is muc more effecient.
        self.oList = np.empty(self.oListSize, dtype = DataObj)
        # This is a list of all nodes in the solution states.

    def smallestColumn(self):
        """
        Takes the root node and finds the column with the least number of nodes in line with Knuth's algorithm.  Note that
        this does not return a copy of the minimum column but actually the minimum column object.  Furthermore the algorithm
        works even if two columns both equal the minimum size by returning the first one encountred.

        for each j ← R[h], R[R[h]], ... , while j != h,
            if S[j] < s set c ← j and s ← S[j].

        The algorithm from Dancing Links.
        """
        size = self.maxColumnSize # This is to ensure that the algorithm works correctly (Knuth uses infinity but well umm)
        root = self.listObj.nodes[0,0]
        column = root.r # column = j, root = h, size = s
        while column.name != root.name:
            if column.size < size:
                columnOut = column # columnOut = c
                size = column.size
            column = column.r
        return columnOut
    
    def coverColumn(self, column):
        """
        Covers a column accoriding the knuths algorithm, given the column.
        
        Set L[R[c]] ← L[c] and R[L[c]]← R[c].
        For each i ← D[c], D[D[c]], ... , while i != c,
            for each j ← R[i], R[R[i]], ... , while j != i,
                set U[D[j]] ← U[j], D[U[j]] ← D[j],
                and set S[C[j]] ← S[C[j]] − 1.
        
        The algorithm from Dancing Links.
        """
        column.r.l = column.l
        # Sets the column to the rights left link to the columns left link.
        column.l.r = column.r
        # Sets the column to the lefts right link to the columns right link.
        iNode = column.d # i = iNode. Sets i to the columns down link.
        while iNode.name != column.name:
            jNode = iNode.r # j = jNode. Sets j to iNodes right link.
            while jNode.name != iNode.name:
                jNode.d.u = jNode.u
                # Skips the up link.
                jNode.u.d = jNode.d
                # Skips the down link.
                jNode.c.size = jNode.c.size-1
                # Take one off the size of the column header.
                jNode = jNode.r
                # Moves the jNode right one.
            iNode = iNode.d
            # Moves the iNode down.

    def uncoverColumn(self, column):
        """
        Uncovers a column.
        
        For each i = U[c], U[U[c]], ... , while i != c,
            for each j ← L[i], L[L[i]], ... , while j != i,
                set S[C[j]] ← S[(C?)[j]] + 1,
                and set U[D[j]] ← j, D[U[j]] ← j.
        Set L[R[c]] ← c and R[L[c]] ← c.

        The algorithm from Dancing Links.
        """
        iNode = column.u
        # i = iNode. Sets i to the columns up link.
        while iNode.name != column.name:
            jNode = iNode.l
            # j = jNode. Sets j to iNodes left link.
            while jNode.name != iNode.name:
                jNode.c.size = jNode.c.size+1
                # Increases jNodes column size by one.
                jNode.d.u = jNode
                # Readds the up link.
                jNode.u.d = jNode
                # Readds the down link
                jNode = jNode.l
                # Moves jNode left one.
            iNode = iNode.u
            # Moves iNode up one.
        column.r.l = column
        column.l.r = column
        # Readds the column links.
    
    def printSolution(self):
        """
        Prints the solution from oList, the list of solution nodes.
        """
        for i in range(self.oListSize):
            if self.oList[i] != None:
                row = str(self.oList[i].c.name)+" "
                iNode = self.oList[i].r
                while iNode.name != self.oList[i].name:
                    row = row+str(iNode.c.name)+" "
                    iNode = iNode.r
                print(row)
        
    def printRows(self):
        """
        Prints the solution from oList, rows of solution nodes.
        """
        count = 0
        for i in range(self.oListSize):
            if self.oList[i] != None:
                row = self.oList[i].row-1
                # Minus one as row index differently.
                print(canLookUp[row]+1) # requires a global variable.
                count = count+1
        print(count)

    def saveRows(self, oList, canLookUp):
        """
        Prints the solution from oList, rows of solution nodes.
        """
        count = 0
        for i in range(self.oListSize):
            if self.oList[i] != None:
                row = self.oList[i].row-1
                # Minus one as row index differently.
                oList.append(canLookUp[row])
                count = count+1

    def search(self, k, oList, canLookUp): # ughhhh recursion
        """
        Solving using Dancing Links.  This is the core function that actually does the solving.  In essence it is a simple
        depth first tree search over the exact cover problem space, however it is implemented in a very computationally
        effecient manner.  Also important to note is that it reduces the searches it does by choosing the column with the
        least constraints, as this frees up constraints later, meaning a solution is more likely.
        
        If R[h] = h, print the current solution (see below) and return
        Otherwise choose a column object c (see below).
        Cover column c (see below).
        For each r ← D[c], D[D[c]], ... , while r != c,
            set Ok ← r;
            for each j ← R[r], R[R[r]], ... , while j != r
                cover column j (see below);
            search(k + 1);
            set r ← Ok and c ← C[r];
            for each j ← L[r], L[L[r]], ... , while j 6= r
                uncover column j (see below).
        Uncover column c (see below) and return.
        
        The algorithm from Dancing Links.
        """
        if self.listObj.nodes[0,0].r == self.listObj.nodes[0,0]:
            # If there are no more columns then a solution has been found.
            
            #print("Solution")
            #self.printSolution()
            #self.printRows()
            #print()
            # Prints solution.
            
            self.saveRows(oList, canLookUp)
            return 0 # passes!
            # Base case for recursion, what it returns is irrelevant.
        else:
            column = self.smallestColumn()
            # Chooses a column.
            self.coverColumn(column)
            # Covers column.
            
            rNode = column.d
            # r ← D[c], sets rNode to the node down from the column header.
            while rNode.name != column.name:
                oNode = rNode
                # Ok = oNode.
                self.oList[k] = oNode
                # Adds the oNode to the oList.
                jNode = rNode.r
                # j ← R[r], sets jNode to the node right of rNode.
                while jNode.name != rNode.name:
                    self.coverColumn(jNode.c)
                    # Covers the coumn of j.
                    jNode = jNode.r
                    # Moves j node one to the right.
                self.search(k+1, oList, canLookUp)
                # Recursion on function to go deeper into tree.
                rNode = oNode
                # Set r ← Ok.
                column = rNode.c
                # Set c ← C[r].
                jNode = rNode.l
                # j ← L[r], sets jNode to the node left of rNode.
                while jNode.name != rNode.name:
                    self.uncoverColumn(jNode.c)
                    # Uncovers the column of j.
                    jNode = jNode.l
                    # Moves j node one to the left.
                rNode = rNode.d
                # Moves rNode one down.
            self.uncoverColumn(column)
            return 1 # No solution...



class SudokuGen:
    """
    The puropose of this class is to create the exact cover matrix for Suduko...  It's huge and terrifying.
    """
    def __init__(self):
        self.ecMat = np.zeros((729, 324), dtype = int)
        # This is the exact cover matrix.
        self.canLookUp = np.zeros((729,3), dtype = int)
        # This is the candidate or row look up table allowing identification of a row from it's index in the matrix.
    
    def saveMats(self):
        """
        Prints the ECMat to a txt file to verify it's validity.
        """
        np.savetxt('ecMat.txt', self.ecMat, delimiter=',', fmt='%d', newline='\n')
        np.savetxt('lookUp.txt', self.canLookUp, delimiter=',', fmt='%d', newline='\n')

    def genCanLookUp(self):
        """
        Generates the row look up table.  This is an array of three values, the first is the row, the second is the
        column and the third is the number.  The purpose of this is such that rows or candidates can be adressed quickly
        by other pieces of the program.
        """
        count = 0
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    self.canLookUp[count] = [i,j,k]
                    count = count+1
    
    def genConstraints(self):
        """
        Puts a cell constraint in the table, given a row's index.  This is what generates the constraints.  The table is an
        auto generated replica of that done by Bob Hanson of St. Olaf College.  The table can be found here,
        https://www.stolaf.edu/people/hansonr/sudoku/exactcovermatrix.htm
        """
        for i in range(729):
            canRow = self.canLookUp[i][0]
            canCol = self.canLookUp[i][1]
            canNum = self.canLookUp[i][2]
        
            self.ecMat[i, canRow*9+canCol] = 1
            # Cells
            self.ecMat[i, 81+canNum+canRow*9] = 1
            # Rows
            self.ecMat[i, 162+canNum+canCol*9] = 1
            # Columns
            self.ecMat[i, (243 + (int(canCol/3)*9) + (int(canRow/3)*27) +canNum)] = 1
            # Blocks
    
    def indexFromCan(self, can):
        """
        Finds the index of a given candidate.
        """
        for i in range(np.shape(self.canLookUp)[0]):
            if np.array_equal(can, self.canLookUp[i]):
                return i
        return False
        # This lot ensures that a row is actually present.
    
    def addConstraint(self, can):
        """
        Adds a constraint given a candidate number by deleting the row, the columns in that row, and any rows in those
        columns.
        """
        row = self.indexFromCan(can)
        if row != False: # to check if a row has already been deleted.
            rows = []
            # Rows to delete.
            columns = []
            # columns to delete.  
            for i in range(np.shape(self.ecMat)[1]):
                if self.ecMat[row,i] == 1:
                    columns.append(i)
                    for j in range(np.shape(self.ecMat)[0]):
                        if self.ecMat[j,i] == 1:
                            rows.append(j)
            # This searches for all columns in the row, and all rows attatched to these columns   
            self.ecMat = np.delete(self.ecMat, columns, axis=1)
            # Deletss the columns
            self.ecMat = np.delete(self.ecMat, rows, axis=0)
            self.canLookUp = np.delete(self.canLookUp, rows, axis=0)
            # Deletes rows

######################################################### Functions ########################################################

def sudoku_solver(array):
    """
    This is the sudoku solving fucntion.  It wraps all classes into one single class.  This is the one to test!!
    
    Note that this could be even faster, every time that the function is run it initialises the eact cover matrix and
    nodes for the exact cover matrix - this is comparitavely slow and on average takes 0.09 seconds on a uni pc, this is
    in contrast to 0.01 seconds to actually solve the sudoku.  Had I programmed the constraint adding better, I probably
    could have only initialised the exact cover matrix once, eliminating setup time for each sudoku.  A 0.01 second
    function compared to a 0.1 second function, well what does it matter.
    
    Timing the entire solve of 20 takes about 2 seconds on a rubbish uni pc so should be vastly under the 30 seconds for
    allowed for each one.
    """
    array = np.array(array, dtype = int)
    # This converts the data type of the array to int, needed for correct operation
    sudokuGen = SudokuGen()
    # Initialises the eaxct cover matrix and look up table.
    sudokuGen.genCanLookUp()
    # Creates the look up table.
    sudokuGen.genConstraints()
    # Creates the eaxct cover matrix. 
    for i in range(9):
        for j in range(9):
            if array[i,j] != 0:
                sudokuGen.addConstraint((i,j,array[i,j]-1))
    # Adds all constraints.
    canLookUp = sudokuGen.canLookUp
    # Makes a local version of the look up table.
    oList = []
    # Creates local output list.
    sudokuSolver = Solver(sudokuGen.ecMat)
    # Initialises the solver.
    sudokuSolver.search(0, oList, canLookUp)
    # Solves the sudoku.
    arrayOut = np.zeros((9,9), dtype = int)
    for i in oList:
        arrayOut[i[0],i[1]] = i[2]+1
    arrayOut = arrayOut+array
    # Copies the informationfrom the output list into an output array.
    if np.array_equal(array,arrayOut):
        arrayOut = np.zeros((9,9), dtype = int)-1
    # If the search was unsucesfull, i.e. the sudoku is unsolveable meaning there are no solutions in the output list and
    # hence the output array matches the input array.  This means the output array is set to minus ones.
    arrayOut = np.array(arrayOut, dtype = float)
    # Chucks the arrayOut into a float format to match what will be expected by the auto marker.
    return(arrayOut)
    

########################################################## Testing #########################################################

"""
time0 = time.time()
# Starts the timer.
for i in range(20):
    array = sudoku_solver(sudokus[i])
    print(np.array_equal(solutions[i],array))
    # Prints true if solved array = solution array.
time1 = time.time()
# Ends the timer
deltatime = time1-time0
# calculates the time difference.
print(deltatime)
"""

'\ntime0 = time.time()\n# Starts the timer.\nfor i in range(20):\n    array = sudoku_solver(sudokus[i])\n    print(np.array_equal(solutions[i],array))\n    # Prints true if solved array = solution array.\ntime1 = time.time()\n# Ends the timer\ndeltatime = time1-time0\n# calculates the time difference.\nprint(deltatime)\n'

## Testing your code

You can test your code on the sudoku puzzles that we have provided in the following cell. This will work only if all of your code is above the test cell. Otherwise, the test cell does not have access to the `sudoku_solver()` function. Before you submit, please comment out any code that you used to test your function on the training puzzles.

In [3]:
# Uncomment the following block to test your code. Comment it out again before submitting.

#for i in range(len(sudokus)):
#    sudoku = sudokus[i].copy()
#    print("This is sudoku number", i)
#    print(sudoku)
#    your_solution = sudoku_solver(sudokus[i])
#    print("This is your solution for sudoku number", i)
#    print(your_solution)
#    print("Is your solution correct?")
#    print(np.array_equal(your_solution, solutions[i]))


## How we will test your code

We will test your code using the hidden tests in the following cell. Specifically, the hidden tests will test your `sudoku_solver()` function on a different set of 20 sudoku puzzles of similar difficulty. **Make sure all of your code is above our test cell. ** Otherwise, the test cell will not have access to the sudoku_solver() function and you will receive zero marks.

## IMPORTANT: How to submit

If any of the following instructions is not clear, please ask your tutors well ahead of the submission deadline.

#### Before you submit
- Restart the kernel (_Kernel $\rightarrow$ Restart & Run All_) and make sure that you can run all cells from top to bottom without any errors.
- Make sure that the test cell has access to the `sudoku_solver()` function that you defined and make sure that this function returns the solved Sudoku in the correct data type and shape.
- Please comment out any code that you used to test your function on the training puzzles.
- Make sure that your code is written in Python 3 (and not in Python 2!). You can check the Python version of the current session in the top-right corner below the Python logo.

#### Submission file
- Please upload to Moodle a single Jupyter notebook file called "ai3_sudoku.ipynb". Do __not__ compress/zip your Jupyter notebook file.
- Do not include __any__ identifying information. Marking is anonymous.

In [4]:
# This is a TEST CELL. Do not delete or change. All of your code must be written above this cell. 

In [5]:
# This is a TEST CELL. Do not delete or change. 

In [6]:
# This is a TEST CELL. Do not delete or change. 

In [7]:
# This is a TEST CELL. Do not delete or change. 

In [8]:
# This is a TEST CELL. Do not delete or change. 

In [9]:
# This is a TEST CELL. Do not delete or change. 

In [10]:
# This is a TEST CELL. Do not delete or change. 

In [11]:
# This is a TEST CELL. Do not delete or change. 

In [12]:
# This is a TEST CELL. Do not delete or change. 

In [13]:
# This is a TEST CELL. Do not delete or change. 

In [14]:
# This is a TEST CELL. Do not delete or change. 

In [15]:
# This is a TEST CELL. Do not delete or change. 

In [16]:
# This is a TEST CELL. Do not delete or change. 

In [17]:
# This is a TEST CELL. Do not delete or change. 

In [18]:
# This is a TEST CELL. Do not delete or change. 

In [19]:
# This is a TEST CELL. Do not delete or change. 

In [20]:
# This is a TEST CELL. Do not delete or change. 

In [21]:
# This is a TEST CELL. Do not delete or change. 

In [22]:
# This is a TEST CELL. Do not delete or change. 

In [23]:
# This is a TEST CELL. Do not delete or change. 

In [24]:
# This is a TEST CELL. Do not delete or change. 